# **MIP4-Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7 """

In [7]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [8]:
print(response1.choices[0].message.content)

Parameters:
t = time step (1 to 10)
f(t) = max(-4/30t^3+t^2-0.234t+3, 0) : Function depicting water entering the system
c_ij : Capacity of pipe G_ij connecting points i and j in the network. According to the problem, c_AB=3, c_AC=6, c_AF=1, c_BC=3, c_BD=10, c_BE=4, c_CE=4, c_CF=4, c_DE=5, c_EG=5, c_FB=12, c_FG=7 (where i,j ∈ {A, B, C, D, E, F, G})

Decision Variables:
Let's denote x_ij(t) as the amount of water that flows from point i to point j at time step t (where i,j ∈ {A,B,C,D,E,F,G})

Objective Function:
We want to maximize the water that flows to the point G over the time. So:

Maximize ∑_t[x_AG(t) + x_BG(t) + x_CG(t) + x_DG(t) + x_EG(t) + x_FG(t)], for t from 1 to 10.

Constraints:

1. Conservation conditions regarding water coming in and out of each point:

a. ∑_j[x_ij(t)] = f(t), for i=A
b. ∑_j[x_ij(t)] = ∑_i[x_ji(t-1)], for i ≠ A, G, and t from 2 to 10

2. Each pipe capacity constraint:

x_ij(t) ≤ c_ij, for all i,j at each time step t

3. No storage condition at each point: 

### **Generate Pyomo Code**

In [9]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [11]:
Markdown(response2.choices[0].message.content)

Here is how you can formulate this problem in Python with the Pyomo library. As the problem described a network has been considered like a directed graph with 7 nodes and 12 edges. Here, I've created a random network based on this. 

```python
from pyomo.environ import *

model = ConcreteModel()

# data
t_max = 10
Nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Edges = [('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')]
Capacity = {('A', 'B'): 3, ('A', 'C'): 6, ('A', 'F'): 1, ('B', 'C'): 3, ('B', 'D'): 10, ('B', 'E'): 4, ('C', 'E'): 4, ('C', 'F'): 4, ('D', 'E'): 5, ('E', 'G'): 5, ('F', 'B'): 12, ('F', 'G'): 7}
M = 1000 # large number

# sets
model.T = RangeSet(t_max)
model.Nodes = Set(initialize=Nodes)
model.Edges = Set(within=model.Nodes*model.Nodes, initialize=Edges)

# vars
model.Flow = Var(model.Edges, model.T, within=NonNegativeReals)
model.z = Var(model.Edges, within=Binary)

# objective
model.obj = Objective(expr=sum(model.Flow[i, j, t] for i, j in model.Edges for t in model.T if j == 'G'), sense=maximize)

# constraints
def inflow_balance(model, i, t):
    if i=='A':
        return sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges) == max(-4/30*t**3+t**2-0.234*t+3, 0)
    elif i!='G':
        if t > 1:
            return sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges) == sum(model.Flow[j, i, t-1] for j in model.Nodes if (j, i) in model.Edges)
        else:
            return Constraint.Skip
    else: return Constraint.Skip

def outflow_balance(model, i, t):
    if i != 'G':
        if t < t_max:
            return sum(model.Flow[j, i, t+1] for j in model.Nodes if (j, i) in model.Edges) == sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges)
        else:
            return Constraint.Skip
    else: return Constraint.Skip

def capacity_rule(model, i, j,t):
    return model.Flow[i,j,t] <= Capacity[i,j]*model.z[i,j]

def pipe_activation_rule(model):
    return sum(model.z[i,j] for i, j in model.Edges) <= 10

def binary_low_bound_rule(model, i, j,t):
    return model.Flow[i,j,t] >= model.z[i,j]

def binary_up_bound_rule(model,i ,j, t):
    return model.Flow[i,j,t] <= M*model.z[i,j]

model.inflow_balance = Constraint(model.Nodes, model.T, rule=inflow_balance)
model.outflow_balance = Constraint(model.Nodes, range(1, t_max), rule=outflow_balance)
model.capacity_rule = Constraint(model.Edges, model.T , rule=capacity_rule)
model.pipe_activation_rule = Constraint(rule=pipe_activation_rule)
model.binary_low_bound_rule = Constraint(model.Edges, model.T, rule=binary_low_bound_rule)
model.binary_up_bound_rule = Constraint(model.Edges, model.T , rule=binary_up_bound_rule)
```
Before running the model, you need to install a solver. You can install the glpk solver by using the following command in the terminal:

`apt-get install -y glpk-utils` 

Then you can solve it by:

```python
# Solve the model
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()
```

Please be sure to replace the nodes and edges according to your specific problem.


### **Run the code Generated by GPT4**

In [13]:
from pyomo.environ import *

model = ConcreteModel()

# data
t_max = 10
Nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Edges = [('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')]
Capacity = {('A', 'B'): 3, ('A', 'C'): 6, ('A', 'F'): 1, ('B', 'C'): 3, ('B', 'D'): 10, ('B', 'E'): 4, ('C', 'E'): 4, ('C', 'F'): 4, ('D', 'E'): 5, ('E', 'G'): 5, ('F', 'B'): 12, ('F', 'G'): 7}
M = 1000 # large number

# sets
model.T = RangeSet(t_max)
model.Nodes = Set(initialize=Nodes)
model.Edges = Set(within=model.Nodes*model.Nodes, initialize=Edges)

# vars
model.Flow = Var(model.Edges, model.T, within=NonNegativeReals)
model.z = Var(model.Edges, within=Binary)

# objective
model.obj = Objective(expr=sum(model.Flow[i, j, t] for i, j in model.Edges for t in model.T if j == 'G'), sense=maximize)

# constraints
def inflow_balance(model, i, t):
    if i=='A':
        return sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges) == max(-4/30*t**3+t**2-0.234*t+3, 0)
    elif i!='G':
        if t > 1:
            return sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges) == sum(model.Flow[j, i, t-1] for j in model.Nodes if (j, i) in model.Edges)
        else:
            return Constraint.Skip
    else: return Constraint.Skip

def outflow_balance(model, i, t):
    if i != 'G':
        if t < t_max:
            return sum(model.Flow[j, i, t+1] for j in model.Nodes if (j, i) in model.Edges) == sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges)
        else:
            return Constraint.Skip
    else: return Constraint.Skip

def capacity_rule(model, i, j,t):
    return model.Flow[i,j,t] <= Capacity[i,j]*model.z[i,j]

def pipe_activation_rule(model):
    return sum(model.z[i,j] for i, j in model.Edges) <= 10

def binary_low_bound_rule(model, i, j,t):
    return model.Flow[i,j,t] >= model.z[i,j]

def binary_up_bound_rule(model,i ,j, t):
    return model.Flow[i,j,t] <= M*model.z[i,j]

model.inflow_balance = Constraint(model.Nodes, model.T, rule=inflow_balance)
model.outflow_balance = Constraint(model.Nodes, range(1, t_max), rule=outflow_balance)
model.capacity_rule = Constraint(model.Edges, model.T , rule=capacity_rule)
model.pipe_activation_rule = Constraint(rule=pipe_activation_rule)
model.binary_low_bound_rule = Constraint(model.Edges, model.T, rule=binary_low_bound_rule)
model.binary_up_bound_rule = Constraint(model.Edges, model.T , rule=binary_up_bound_rule)


# Solve the model
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 470
  Number of variables: 132
  Number of nonzeros: 1131
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.006440162658691406


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [14]:
from pyomo.environ import *

model = ConcreteModel()

# data
t_max = 10
Nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Edges = [('A', 'B'), ('A', 'C'), ('A', 'F'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'E'), ('C', 'F'), ('D', 'E'), ('E', 'G'), ('F', 'B'), ('F', 'G')]
Capacity = {('A', 'B'): 3, ('A', 'C'): 6, ('A', 'F'): 1, ('B', 'C'): 3, ('B', 'D'): 10, ('B', 'E'): 4, ('C', 'E'): 4, ('C', 'F'): 4, ('D', 'E'): 5, ('E', 'G'): 5, ('F', 'B'): 12, ('F', 'G'): 7}
M = 1000 # large number

# sets
model.T = RangeSet(t_max)
model.Nodes = Set(initialize=Nodes)
model.Edges = Set(within=model.Nodes*model.Nodes, initialize=Edges)

# vars
model.Flow = Var(model.Edges, model.T, within=NonNegativeReals)
model.z = Var(model.Edges, within=Binary)

# objective
model.obj = Objective(expr=sum(model.Flow[i, j, t] for i, j in model.Edges for t in model.T if j == 'G'), sense=maximize)

# constraints
def inflow_balance(model, i, t):
    if i=='A':
        return sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges) == max(-4/30*t**3+t**2-0.234*t+3, 0)
    elif i!='G':
        if t > 1:
            return sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges) == sum(model.Flow[j, i, t-1] for j in model.Nodes if (j, i) in model.Edges)
        else:
            return Constraint.Skip
    else: return Constraint.Skip

def outflow_balance(model, i, t):
    if i != 'G':
        if t < t_max:
            return sum(model.Flow[j, i, t+1] for j in model.Nodes if (j, i) in model.Edges) == sum(model.Flow[i, j, t] for j in model.Nodes if (i, j) in model.Edges)
        else:
            return Constraint.Skip
    else: return Constraint.Skip

def capacity_rule(model, i, j,t):
    return model.Flow[i,j,t] <= Capacity[i,j]

def pipe_activation_rule(model):
    return sum(model.z[i,j] for i, j in model.Edges) <= 10

def binary_low_bound_rule(model, i, j,t):
    return model.Flow[i,j,t] >= model.z[i,j]

def binary_up_bound_rule(model,i ,j, t):
    return model.Flow[i,j,t] <= M*model.z[i,j]

model.inflow_balance = Constraint(model.Nodes, model.T, rule=inflow_balance)
model.outflow_balance = Constraint(model.Nodes, range(1, t_max), rule=outflow_balance)
model.capacity_rule = Constraint(model.Edges, model.T , rule=capacity_rule)
model.pipe_activation_rule = Constraint(rule=pipe_activation_rule)
model.binary_low_bound_rule = Constraint(model.Edges, model.T, rule=binary_low_bound_rule)
model.binary_up_bound_rule = Constraint(model.Edges, model.T , rule=binary_up_bound_rule)


# Solve the model
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 470
  Number of variables: 132
  Number of nonzeros: 1011
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.011562585830688477
